In [1]:
import dash
from dash.dependencies import Input, Output
import dash_bio as dashbio
from dash import html, dcc

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    'Select which chromosomes to display on the ideogram below:',
#    dcc.Dropdown(
#        id='dash-chromosomes',
#        options=[{'label': str(i), 'value': str(i)} for i in range(1, 23)],
#        multi=True,
#        value=[str(i) for i in range(1, 23)]
#    ),
    dashbio.Ideogram(
        id='dashbio-ideogram',
        chrHeight=600,
        chrWidth=20,
        dataDir='https://aedawid.github.io/database/',
        organism="custom-2",
        annotationsPath='https://eweitz.github.io/ideogram/data/annotations/SRR562646.json',
        chromosomes=['X', 'Y'],
#        rotatable=True,
#        orientation='vertical',
#        brush='chrX:1-10000000'
    ),
    html.Div(id='my-default-ideogram-rotated')
])

@app.callback(
    Output('dashbio-ideogram', 'chromosomes'),
    Input('dash-chromosomes', 'value')
)
def update_ideogram(value):
    return value

app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2022 16:35:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 16:35:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 16:35:03] "GET /_dash-dependencies HTTP/1.1" 200 -
